### Load libraries

In [1]:
from warnings import simplefilter

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (f1_score, precision_score, recall_score)
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import (Pipeline, make_pipeline)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier

simplefilter(action='ignore', category=FutureWarning)
simplefilter("always", ConvergenceWarning)

### Load data

In [2]:
data = pd.read_csv('Train_nyOWmfK.csv', encoding='latin1')

### Data informations, statistics and describe

In [3]:
data.shape

(87020, 26)

In [4]:
data.head(10)

ID  Gender        City  Monthly_Income        DOB  \
0  ID000002C20  Female       Delhi           20000  23-May-78   
1  ID000004E40    Male      Mumbai           35000  07-Oct-85   
2  ID000007H20    Male   Panchkula           22500  10-Oct-81   
3  ID000008I30    Male     Saharsa           35000  30-Nov-87   
4  ID000009J40    Male   Bengaluru          100000  17-Feb-84   
5  ID000010K00    Male   Bengaluru           45000  21-Apr-82   
6  ID000011L10  Female  Sindhudurg           70000  23-Oct-87   
7  ID000012M20    Male   Bengaluru           20000  25-Jul-75   
8  ID000013N30    Male       Kochi           75000  26-Jan-72   
9  ID000014O40  Female      Mumbai           30000  12-Sep-89   

  Lead_Creation_Date  Loan_Amount_Applied  Loan_Tenure_Applied  Existing_EMI  \
0          15-May-15             300000.0                  5.0           0.0   
1          04-May-15             200000.0                  2.0           0.0   
2          19-May-15             600000.0                  4.0           0.0   
3          09-May-15            1000000.0                  5.0           0.0   
4          20-May-15             500000.0                  2.0       25000.0   
5          20-May-15             300000.0                  5.0       15000.0   
6          01-May-15                  6.0                  5.0           0.0   
7          20-May-15             200000.0                  5.0        2597.0   
8          02-May-15                  0.0                  0.0           0.0   
9          03-May-15             300000.0                  3.0           0.0   

                                  Employer_Name  ... Interest_Rate  \
0                                       CYBOSOL  ...           NaN   
1           TATA CONSULTANCY SERVICES LTD (TCS)  ...         13.25   
2                       ALCHEMIST HOSPITALS LTD  ...           NaN   
3                              BIHAR GOVERNMENT  ...           NaN   
4                          GLOBAL EDGE SOFTWARE  ...           NaN   
5  COGNIZANT TECHNOLOGY SOLUTIONS INDIA PVT LTD  ...         13.99   
6                          CARNIVAL CRUISE LINE  ...           NaN   
7               GOLDEN TULIP FLORITECH PVT. LTD  ...           NaN   
8                                  SIIS PVT LTD  ...         14.85   
9                                SOUNDCLOUD.COM  ...         18.25   

  Processing_Fee  EMI_Loan_Submitted Filled_Form  Device_Type  Var2  Source  \
0            NaN                 NaN           N  Web-browser     G    S122   
1            NaN             6762.90           N  Web-browser     G    S122   
2            NaN                 NaN           N  Web-browser     B    S143   
3            NaN                 NaN           N  Web-browser     B    S143   
4            NaN                 NaN           N  Web-browser     B    S134   
5         1500.0             6978.92           N  Web-browser     B    S143   
6            NaN                 NaN           N  Web-browser     B    S133   
7            NaN                 NaN           N  Web-browser     B    S159   
8        26000.0            30824.65           Y       Mobile     C    S122   
9         1500.0            10883.38           N  Web-browser     B    S133   

   Var4  LoggedIn Disbursed  
0     1         0         0  
1     3         0         0  
2     1         0         0  
3     3         0         0  
4     3         1         0  
5     3         1         0  
6     1         0         0  
7     3         0         0  
8     5         0         0  
9     1         0         0  

[10 rows x 26 columns]

In [5]:
data.dtypes

ID                        object
Gender                    object
City                      object
Monthly_Income             int64
DOB                       object
Lead_Creation_Date        object
Loan_Amount_Applied      float64
Loan_Tenure_Applied      float64
Existing_EMI             float64
Employer_Name             object
Salary_Account            object
Mobile_Verified           object
Var5                       int64
Var1                      object
Loan_Amount_Submitted    float64
Loan_Tenure_Submitted    float64
Interest_Rate            float64
Processing_Fee           float64
EMI_Loan_Submitted       float64
Filled_Form               object
Device_Type               object
Var2                      object
Source                    object
Var4                       int64
LoggedIn                   int64
Disbursed                  int64
dtype: object

In [6]:
data.describe(percentiles=[0.2, 0.4, 0.6, 0.8])

Monthly_Income  Loan_Amount_Applied  Loan_Tenure_Applied  Existing_EMI  \
count    8.702000e+04         8.694900e+04         86949.000000  8.694900e+04   
mean     5.884997e+04         2.302507e+05             2.131399  3.696228e+03   
std      2.177511e+06         3.542068e+05             2.014193  3.981021e+04   
min      0.000000e+00         0.000000e+00             0.000000  0.000000e+00   
20%      1.500000e+04         0.000000e+00             0.000000  0.000000e+00   
40%      2.060920e+04         1.000000e+05             1.000000  0.000000e+00   
50%      2.500000e+04         1.000000e+05             2.000000  0.000000e+00   
60%      2.800000e+04         2.000000e+05             3.000000  0.000000e+00   
80%      4.500000e+04         3.000000e+05             5.000000  5.000000e+03   
max      4.445544e+08         1.000000e+07            10.000000  1.000000e+07   

               Var5  Loan_Amount_Submitted  Loan_Tenure_Submitted  \
count  87020.000000           5.240700e+04           52407.000000   
mean       4.961503           3.950106e+05               3.891369   
std        5.670385           3.082481e+05               1.165359   
min        0.000000           5.000000e+04               1.000000   
20%        0.000000           1.900000e+05               3.000000   
40%        1.000000           2.900000e+05               4.000000   
50%        2.000000           3.000000e+05               4.000000   
60%        3.000000           3.700000e+05               4.000000   
80%       11.000000           5.000000e+05               5.000000   
max       18.000000           3.000000e+06               6.000000   

       Interest_Rate  Processing_Fee  EMI_Loan_Submitted          Var4  \
count   27726.000000    27420.000000        27726.000000  87020.000000   
mean       19.197474     5131.150839        10999.528377      2.949805   
std         5.834213     4725.837644         7512.323050      1.697720   
min        11.990000      200.000000         1176.410000      0.000000   
20%        14.850000     2000.000000         5422.580000      1.000000   
40%        16.350000     3000.000000         8342.710000      3.000000   
50%        18.000000     4000.000000         9392.970000      3.000000   
60%        18.400000     4400.000000        10597.550000      3.000000   
80%        20.000000     7500.000000        14362.930000      5.000000   
max        37.000000    50000.000000       144748.280000      7.000000   

           LoggedIn     Disbursed  
count  87020.000000  87020.000000  
mean       0.029350      0.014629  
std        0.168785      0.120062  
min        0.000000      0.000000  
20%        0.000000      0.000000  
40%        0.000000      0.000000  
50%        0.000000      0.000000  
60%        0.000000      0.000000  
80%        0.000000      0.000000  
max        1.000000      1.000000

### train-test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["LoggedIn", "Disbursed"], axis=1),
                                          data.Disbursed,
                                          test_size=10000,
                                          random_state=123)

### data preparation for machine learning, pre-processing and processing data  by type

In [8]:
def  Preproces(data):
    data = data.drop(["ID", "Lead_Creation_Date", "Employer_Name", "City", "Salary_Account"], axis=1)
    data["Gender"].replace({'Female': 0, 'Male':1}, inplace = True)
    data["Mobile_Verified"].replace({'Y': 0, 'N':1}, inplace = True)
    data["Filled_Form"].replace({'Y': 0, 'N':1}, inplace = True)
    data["Device_Type"].replace({'Web-browser': 0, 'Mobile':1}, inplace = True)
    data["DOB"] = 2015 -('19' + data["DOB"].str[-2:]).astype('int')
    
    return data

def Proces():
    num_features = ["Monthly_Income", "DOB", "Loan_Amount_Applied", "Loan_Tenure_Applied", "Existing_EMI",
                   "Loan_Amount_Submitted", "Loan_Tenure_Submitted", "Interest_Rate", "Processing_Fee",
                   "EMI_Loan_Submitted", "Var5"]
    num_pipeline = make_pipeline(SimpleImputer(strategy = "constant", fill_value = 0), 
                                 StandardScaler())
    cat_feauters = ["Var1", "Var2", "Var4", "Source"]
    cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),
                                OneHotEncoder(handle_unknown="ignore"))
    other_feauters = ["Gender", "Mobile_Verified", "Filled_Form", "Device_Type"]
    other_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"))

    processor = ColumnTransformer(transformers=[
        ("num", num_pipeline, num_features),
        ("cat", cat_pipeline, cat_feauters),
        ("cabin", other_pipeline, other_feauters)
    ])
    return processor

In [9]:
X_test = Preproces(X_test)
X_train = Preproces(X_train)

### machine learning process pipeline with two models and a grid search including data imbalance

In [21]:
num_splits = 10

param_grid_lr =  [{
    'penalty': ['l2'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
    'class_weight': ['balanced']
    },
    {
    'penalty': ['l1'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['liblinear', 'saga'],
    'class_weight': ['balanced']
    },
    {
    'penalty': ['elasticnet'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['saga'],
    'class_weight': ['balanced']
    }
]
param_grid_rf =  [{
    'n_estimators' : list(range(61,101,15)),
    'max_features' : list(range(12,53,15)),
    'min_samples_split': list(range(500,1103,150)),
    'class_weight': ['balanced', 'balanced_subsample']
    }]

kf = KFold(n_splits=num_splits)
grid_lr = GridSearchCV(LogisticRegression(), param_grid=param_grid_lr, 
                       cv=kf, verbose=10, refit=True,  n_jobs=-1)
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid_rf, 
                       cv=kf, verbose=10, refit=True, n_jobs=-1)
                                 
classification_process_logistic_regression = make_pipeline(
    Proces(),
    SelectFromModel(RandomForestClassifier(n_estimators=25, max_features='auto'), 0.0001),
    grid_lr)

classification_process_random_forest = make_pipeline(
    Proces(),
    SelectFromModel(RandomForestClassifier(n_estimators=25, max_features='auto'), 0.0001),
    grid_rf)

In [11]:
classification_process_logistic_regression.fit(X_train, y_train)
classification_process_logistic_regression.predict(X_test)

Fitting 10 folds for each of 70 candidates, totalling 700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  4

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)

In [15]:
print(precision_score(y_test, classification_process_logistic_regression.predict_proba(X_test)[:,1]>0.7, 
                      average='macro', zero_division=0))
print(recall_score(y_test, classification_process_logistic_regression.predict_proba(X_test)[:,1]>0.7, 
                   average='macro', zero_division=0))
print(f1_score(y_test, classification_process_logistic_regression.predict_proba(X_test)[:,1]>0.7,
               average='macro', zero_division=0))

0.526307029096567
0.6917680379080985
0.5208957958281591


In [22]:
classification_process_random_forest.fit(X_train, y_train)
classification_process_random_forest.predict(X_test)

Fitting 10 folds for each of 90 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 20.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 25.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 41

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [23]:
print(precision_score(y_test, classification_process_random_forest.predict_proba(X_test)[:,1]>0.7,
                       average='macro', zero_division=0))
print(recall_score(y_test, classification_process_random_forest.predict_proba(X_test)[:,1]>0.7,
                   average='macro', zero_division=0))
print(f1_score(y_test, classification_process_random_forest.predict_proba(X_test)[:,1]>0.7,
               average='macro', zero_division=0))

0.5323147412903991
0.6245503922431503
0.5432502902179572


### for comparison, the same pipline with the wrong assumption or ignorance that the data is unbalanced

In [26]:
num_splits = 10

param_grid_lr =  [{
    'penalty': ['l2'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
    'class_weight': [None]
    },
    {
    'penalty': ['l1'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['liblinear', 'saga'],
    'class_weight': [None]
    },
    {
    'penalty': ['elasticnet'],
    'C' : np.logspace(-2.5, 0.7, 10),
    'solver' : ['saga'],
    'class_weight': [None]
    }
]
param_grid_rf =  [{
    'n_estimators' : list(range(61,101,15)),
    'max_features' : list(range(12,53,15)),
    'min_samples_split': list(range(500,1103,150)),
    'class_weight': [None]
    }]

kf = KFold(n_splits=num_splits)
grid_lr = GridSearchCV(LogisticRegression(), param_grid=param_grid_lr, 
                       cv=kf, verbose=10, refit=True,  n_jobs=-1)
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid_rf, 
                       cv=kf, verbose=10, refit=True, n_jobs=-1)
                                 
classification_process_logistic_regression_all = make_pipeline(
    Proces(),
    SelectFromModel(RandomForestClassifier(n_estimators=25, max_features='auto'), 0.0001),
    grid_lr)

classification_process_random_forest_all = make_pipeline(
    Proces(),
    SelectFromModel(RandomForestClassifier(n_estimators=25, max_features='auto'), 0.0001),
    grid_rf)

In [27]:
classification_process_logistic_regression_all.fit(X_train, y_train)
classification_process_logistic_regression_all.predict(X_test)

Fitting 10 folds for each of 70 candidates, totalling 700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  3

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
print(precision_score(y_test, classification_process_logistic_regression_all.predict_proba(X_test)[:,1]>0.7, 
                      average='macro', zero_division=0))
print(recall_score(y_test, classification_process_logistic_regression_all.predict_proba(X_test)[:,1]>0.7, 
                   average='macro', zero_division=0))
print(f1_score(y_test, classification_process_logistic_regression_all.predict_proba(X_test)[:,1]>0.7,
               average='macro', zero_division=0))

0.49235
0.5
0.4961455131757948


In [29]:
classification_process_random_forest_all.fit(X_train, y_train)
classification_process_random_forest_all.predict(X_test)

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 26.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 29.7min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 38

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [30]:
print(precision_score(y_test, classification_process_random_forest_all.predict_proba(X_test)[:,1]>0.7,
                       average='macro', zero_division=0))
print(recall_score(y_test, classification_process_random_forest_all.predict_proba(X_test)[:,1]>0.7,
                   average='macro', zero_division=0))
print(f1_score(y_test, classification_process_random_forest_all.predict_proba(X_test)[:,1]>0.7,
               average='macro', zero_division=0))

0.49235
0.5
0.4961455131757948


### Conclusion
For unbalanced data, balancing or choosing methods that can oppose this is important, as shown in the attached example.
Both methods, after taking into account the balance between classes, came out better than without taking into account. The methods without balancing were below 50% prediction.
The method is more resistant (by itself more resistant) to such a problem, the random forest did better in this case, obtaining f1 score 54%.